<a href="https://colab.research.google.com/github/Martin-Roberto-lecuona/concurrente-M1/blob/main/TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Directorios

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!mkdir TP1
!mkdir TP1/Parte-1
!mkdir TP1/Parte-2
!mkdir TP1/Parte-3

# Codigo en C

##Parte 1

#### Crear el archivo

In [ ]:
%%writefile TP1/Parte-1/tp1_p1_c++.cpp
#include <iostream>
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <sys/wait.h>
#include <sys/types.h>
#include <map>
#include <vector>

#define SLEEP_TIME 20

using namespace std;

struct ProcesoInfo
{
    char proc;
    pid_t pid;
    pid_t ppid;
};

void imprimir_info(char proc)
{
    ProcesoInfo p = {proc, getpid(), getppid()};
    cout << "Proceso: " << p.proc << ", PID: " << p.pid << ", PPID: " << p.ppid << endl;
}

void imprimir_error(char proc)
{
    cerr << "Error al crear proceso " << proc << endl;
    exit(EXIT_FAILURE);
}

void crear_arbol(const std::pair<const char, std::vector<char>> &p, std::map<char, std::vector<char>> &procesoRelaciones)
{
    char padre = p.first;
    imprimir_info(padre);

    for (auto const &hijo : p.second)
    {
        if (procesoRelaciones.find(hijo) != procesoRelaciones.end())
        {
            // hijo que es padre
            pid_t pid_padre = fork();
            if (pid_padre < 0)
                imprimir_error(padre);
            else if (pid_padre == 0)
            {
                crear_arbol(*procesoRelaciones.find(hijo), procesoRelaciones);
                wait(NULL);
                sleep(SLEEP_TIME);
                exit(EXIT_SUCCESS);
            }
        } else
        {
            // hijo que es hoja
            pid_t pid_hoja = fork();
            if (pid_hoja < 0)
                imprimir_error(hijo);
            else if (pid_hoja == 0)
            {
                imprimir_info(hijo);
                sleep(SLEEP_TIME);
                exit(EXIT_SUCCESS);
            }
        }
    }
}

int main(int argc, char *argv[])
{
    if (argc == 2 && (strcmp(argv[1], "-h") == 0 || strcmp(argv[1], "--help") == 0))
    {
        cout << "Este proceso crea un árbol de procesos" << endl;
        return EXIT_SUCCESS;
    }

    char raiz = 'A';
    map<char, vector<char>> procesoRelaciones =
    {
        {'A', {'B', 'C', 'D'}},
        {'B', {'E', 'F'}},
        {'F', {'H', 'I'}},
        {'I', {'J'}},
        {'D', {'G'}}
    };

    crear_arbol(*procesoRelaciones.find(raiz), procesoRelaciones);

    while (wait(NULL) > 0);

    return EXIT_SUCCESS;
}



Writing TP1/Parte-1/tp1_p1_c++.cpp


Ver si se creo

In [ ]:
!ls -laR | grep tp1_p1_c++.cpp

-rw-r--r-- 1 root root 2166 Apr 12 21:46 tp1_p1_c++.cpp


#### Compilacion

In [ ]:
!g++ -o tp1P1.bin TP1/Parte-1/tp1_p1_c++.cpp

#### Ejecucion en background

Opcional: Help

In [ ]:
!./tp1P1.bin -h

Este proceso crea un árbol de procesos


In [ ]:
!nohup ./tp1P1.bin 1>salidaC 2>/dev/null &

#### Ver arbol de procesos

In [ ]:
!ps -ef | grep tp1P1.bin

root         452       1  0 21:46 ?        00:00:00 ./tp1P1.bin
root         453     452  0 21:46 ?        00:00:00 ./tp1P1.bin
root         454     452  0 21:46 ?        00:00:00 ./tp1P1.bin
root         455     452  0 21:46 ?        00:00:00 ./tp1P1.bin
root         456     453  0 21:46 ?        00:00:00 ./tp1P1.bin
root         457     453  0 21:46 ?        00:00:00 ./tp1P1.bin
root         458     455  0 21:46 ?        00:00:00 ./tp1P1.bin
root         459     457  0 21:46 ?        00:00:00 ./tp1P1.bin
root         460     457  0 21:46 ?        00:00:00 ./tp1P1.bin
root         461     460  0 21:46 ?        00:00:00 ./tp1P1.bin
root         478     225  0 21:46 ?        00:00:00 /bin/bash -c ps -ef | grep tp1P1.bin
root         480     478  0 21:46 ?        00:00:00 grep tp1P1.bin


Cambie el ID del proceso que solo diga ./tp1P1.bin

In [ ]:
!pstree 452

tp1P1.bin─┬─tp1P1.bin─┬─tp1P1.bin
          │           └─tp1P1.bin─┬─tp1P1.bin
          │                       └─tp1P1.bin───tp1P1.bin
          ├─tp1P1.bin
          └─tp1P1.bin───tp1P1.bin


**Salida del programa**

In [ ]:
!cat salidaC

Proceso: A, PID: 452, PPID: 1
Proceso: B, PID: 453, PPID: 452
Proceso: D, PID: 455, PPID: 452
Proceso: F, PID: 457, PPID: 453
Proceso: C, PID: 454, PPID: 452
Proceso: G, PID: 458, PPID: 455
Proceso: H, PID: 459, PPID: 457
Proceso: E, PID: 456, PPID: 453
Proceso: I, PID: 460, PPID: 457
Proceso: J, PID: 461, PPID: 460


**Matar al proceso de ser necesario**

In [ ]:
!kill 22929

## Parte 2

### Creación archivo

In [ ]:
%%writefile TP1/Parte-2/tp1_p2_c++.cpp
#include <chrono>
#include <iostream>
#include <thread>
#include <vector>

#define TAM_MATRIZ 10000
#define CANT_HILOS 2
#define ESCALAR 3

using namespace std;
using namespace std::chrono;

int matrizMult[TAM_MATRIZ][TAM_MATRIZ] = {0};
int matrizRS[TAM_MATRIZ][TAM_MATRIZ] = {0};
int matrizRC[TAM_MATRIZ][TAM_MATRIZ] = {0};

void cargarMatrizMult()
{
  for (int i = 0; i < TAM_MATRIZ; i++)
  {
    for (int j = 0; j < TAM_MATRIZ; j++)
    {
      matrizMult[i][j] = rand() % 10;
    }
  }
}

void cargarMatrizRS()
{
  for (int i = 0; i < TAM_MATRIZ; i++)
  {
    for (int j = 0; j < TAM_MATRIZ; j++)
    {
      matrizRS[i][j] = matrizMult[i][j] * ESCALAR;
    }
  }
}

void cargarFilasRC(int inicio, int fin)
{
  for (int i = inicio; i < fin; i++)
  {
    for (int j = 0; j < TAM_MATRIZ; j++)
    {
      matrizRC[i][j] = matrizMult[i][j] * ESCALAR;
    }
  }
}

void cargarMatrizRC()
{
  vector<thread> hilos;
  int filasPorHilo = TAM_MATRIZ / CANT_HILOS;
  int cantHilosAmpliados =
      TAM_MATRIZ % CANT_HILOS; // Si existe resto, lo repartire entre hilos.
  int inicio;
  int fin = 0;
  // Puede que cantHilosAmpliados sea 0, por lo que no entraria al primer for y
  // fin no tendria valor, y en el segundo for inicio=fin, entonces fin=0 para
  // que inicio empiece desde el principio.

  for (int i = 0; i < cantHilosAmpliados; i++)
  {
    int filasFinales = filasPorHilo + 1;
    inicio = i * filasFinales;
    fin = inicio + filasFinales;

    hilos.push_back(thread(cargarFilasRC, inicio, fin));
  }

  for (int i = cantHilosAmpliados; i < CANT_HILOS; i++)
  {
    inicio = fin;
    fin = inicio + filasPorHilo;

    hilos.push_back(thread(cargarFilasRC, inicio, fin));
  }

  for (int i = 0; i < CANT_HILOS; i++)
  {
    hilos[i].join();
  }

  hilos.clear();
}

void compararMatrices()
{
  bool hayDiferencia = false;

  for (int i = 0; i < TAM_MATRIZ; i++)
  {
    for (int j = 0; j < TAM_MATRIZ; j++)
    {
      int valorRS = matrizRS[i][j];
      int valorRC = matrizRC[i][j];
      if (valorRS != valorRC)
      {
        hayDiferencia = true;
        cout << "Diferencia: RS[" << i << "][" << j << "]=" << valorRS
             << " | RS[" << i << "][" << j << "]=" << valorRC << endl;
      }
    }
  }

  if (!hayDiferencia)
  {
    cout << "Las matrices RS y RC son iguales." << endl;
  }
}

int main(int argc, char const *argv[])
{
  srand(time(NULL));

  cargarMatrizMult();

  cout << "Tamaño de matriz: " << TAM_MATRIZ << " x " << TAM_MATRIZ << endl;

  // Iniciar el reloj
  auto inicio = high_resolution_clock::now();
  // auto le dice al compilador que determine el tipo de la variable en funcion
  // del tipo de expresion con la que se inicializa. Util cuando el tipo del
  // objeto devuelto por una funcion es complejo o dificil de escribir
  // explicitamente. La variable sigue siendo estaticamente tipada, no dinamica.

  // Llamar a la función que deseo medir
  cargarMatrizRS();

  // Detener el reloj
  auto fin = high_resolution_clock::now();

  // Calcular la duración en milisegundos
  auto duracion = duration_cast<milliseconds>(fin - inicio);

  // Imprimir el tiempo transcurrido
  cout << "La función cargarMatrizRS() tardó " << duracion.count()
       << " milisegundos." << endl;

  // Iniciar el reloj
  inicio = high_resolution_clock::now();

  // Llamar a la función que deseo medir
  cargarMatrizRC();

  // Detener el reloj
  fin = high_resolution_clock::now();

  // Calcular la duración en milisegundos
  duracion = duration_cast<milliseconds>(fin - inicio);

  // Imprimir el tiempo transcurrido
  cout << "La función cargarMatrizRC() utilizó " << CANT_HILOS
       << " hilo/s y tardó " << duracion.count() << " milisegundos." << endl;

  compararMatrices();

  return 0;
}

Overwriting TP1/Parte-2/tp1_p2_c++.cpp


### Compilación

In [ ]:
!g++ -o tp1P2.bin TP1/Parte-2/tp1_p2_c++.cpp

### Ejecución

In [ ]:
!./tp1P2.bin

Tamaño de matriz: 10000 x 10000
La función cargarMatrizRS() tardó 514 milisegundos.
La función cargarMatrizRC() utilizó 2 hilo/s y tardó 414 milisegundos.
Las matrices RS y RC son iguales.


## Parte 3

#### Creacion Archivo

In [ ]:
%%writefile TP1/Parte-3/tp1_p3_c++.cpp
#include <iostream>
#include <thread>
#include <vector>
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <sys/types.h>
#include <map>
#include <vector>
#include <fstream>
#include <atomic>
#include <sstream>
#include <chrono>
#include <mutex>
#include <unordered_map>

// Lo que arruina mucho el tiempo es el mapa debido a que hay que usar mutex. Hay bibliotecas como tbb o Folly que tienen
// mapas con una sicronizacion interna, parecido a un atomic pero se complica mucho pasarlo de maquina a maquina por que hay que
// instalar muchas dependencias

using namespace std;
using namespace std::chrono;
// MUTEX
mutex mtx_palabras;
mutex mtx_caracteres;
mutex mtx_espacios;

mutex mtx_mapa;

// Global Vars
int cant_lineas = 0;
atomic<int> cant_palabras(0);
atomic<int> cant_caracteres(0);
atomic<int> cant_espacios(0);

int cant_lineas_mtx = 0;
int cant_palabras_mtx = 0;
int cant_caracteres_mtx = 0;
int cant_espacios_mtx = 0;

unordered_map<string, int> word_count;
unordered_map<string, int> word_count_mtx;

// Funciones
void resolucion_atomic(string file_name);
void creacion_hilos_atomic(fstream *file);
void mostrar_atomic();
void calculo_atomic(string line);
void resolucion_mutex(string file_name);
void creacion_hilos_mutex(fstream *file);
void mostrar_mutex();
void calculo_mutex(string line);

// MAIN
int main(int argc, char const *argv[]){

    string file_name;

    if(argc == 2)
    {
        file_name=argv[1];
        cout <<"RTA :"<< endl;
    }
    else
    {
        cerr << "Falta nombre de archivo"<<endl;
        return -1;
    }

    try
    {
        resolucion_atomic(file_name);
        resolucion_mutex(file_name);

    } catch (const exception &e)
    {
        cerr << "Error durante el cálculo: " << e.what() << endl;
        return -1;
    }

    return 0;
}

void resolucion_atomic(string file_name)
{
    fstream file;
    file.open(file_name, ios::in);
    if (!file)
		cerr << "Error al abrir el archivo";
    cout<<"================= C++ ========================="<<endl;
    auto inicio = high_resolution_clock::now();

    creacion_hilos_atomic(&file);

    auto fin = high_resolution_clock::now();
    auto duracion = duration_cast<milliseconds>(fin - inicio);

    mostrar_atomic();
    cout<<"=========== calculo_con_atomic " << duracion.count() << " ms." "================"<<endl;
    file.close();
}

void creacion_hilos_atomic(fstream *file)
{
    string  line;
    vector<thread> hilos;

    int treads_not_detached = 0;
    while(getline(*file,line))
    {
        hilos.push_back(thread(calculo_atomic, line));
        treads_not_detached++;
        if (treads_not_detached > 10000)
        {
            for (auto &t : hilos)
                if (t.joinable())
                    t.join();
            treads_not_detached = 0;
        }

        cant_lineas++;
    }

    for (auto &t : hilos)
        if (t.joinable())
            t.join();

    file->close();
}

void mostrar_atomic()
{
    cout << "Cantidad de Lineas: " << cant_lineas << endl;
    cout << "Cantidad de Caracteres: " << cant_caracteres << endl;
    cout << "Cantidad de Palabras: " << cant_palabras << endl;
    cout << "Cantidad de Espacios: " << cant_espacios << endl;
    int max_val = 0;
    string max_key;
    for (const auto &p : word_count)
    {
        if (p.second > max_val)
        {
            max_val = p.second;
            max_key = p.first;
        }
    }
    cout << "Palabra con mas repeticion '" << max_key << "', con " << max_val << " repeticiones " << endl;
}

void calculo_atomic(string line)
{
    istringstream iss(line);
    string word;
    while (iss >> word)
    {
         if (word != " ")
         {
            atomic_fetch_add(&cant_palabras, 1);
            mtx_mapa.lock();
            word_count[word]++;
            mtx_mapa.unlock();
         }
    }
    for (char c : line)
        if (c == ' ')
            atomic_fetch_add(&cant_espacios, 1);
        else
            atomic_fetch_add(&cant_caracteres, 1);
}

void resolucion_mutex(string file_name)
{
    fstream file;
    file.open(file_name, ios::in);
    if (!file)
		cerr << "Error al abrir el archivo";
    cout<<"================= C++ ========================="<<endl;
    auto inicio = high_resolution_clock::now();

    creacion_hilos_mutex(&file);

    auto fin = high_resolution_clock::now();
    auto duracion = duration_cast<milliseconds>(fin - inicio);

    mostrar_mutex();
    cout<<"=========== calculo_con_mutex " << duracion.count() << " ms." "================"<<endl;
    file.close();
}

void creacion_hilos_mutex(fstream *file)
{
    string  line;
    vector<thread> hilos;

    while(getline(*file,line))
    {
        hilos.push_back(thread(calculo_mutex, line));
        if (cant_lineas_mtx > 10000)
        {
            for (auto &t : hilos)
                if (t.joinable())
                    t.join();
        }
        cant_lineas_mtx++;
    }

    for (auto &t : hilos)
        if (t.joinable())
            t.join();

    file->close();
}

void mostrar_mutex()
{
    cout << "Cantidad de Lineas: " << cant_lineas_mtx << endl;
    cout << "Cantidad de Caracteres: " << cant_caracteres_mtx << endl;
    cout << "Cantidad de Palabras: " << cant_palabras_mtx << endl;
    cout << "Cantidad de Espacios: " << cant_espacios_mtx << endl;
    int max_val = 0;
    string max_key;
    for (const auto &p : word_count_mtx)
    {
        if (p.second > max_val)
        {
            max_val = p.second;
            max_key = p.first;
        }
    }
    cout << "Palabra con mas repeticion '" << max_key << "', con " << max_val << " repeticiones " << endl;
}

void calculo_mutex(string line) {
    istringstream iss(line);
    string word;

    while (iss >> word) {
        if (word != " ") {
            mtx_palabras.lock();
            cant_palabras_mtx++;
            mtx_palabras.unlock();
            mtx_mapa.lock();
            word_count_mtx[word]++;
            mtx_mapa.unlock();
        }
    }

    for (char c : line) {
        if (c == ' ') {
            mtx_espacios.lock();
            cant_espacios_mtx++;
            mtx_espacios.unlock();
        } else {
            mtx_caracteres.lock();
            cant_caracteres_mtx++;
            mtx_caracteres.unlock();
        }
    }
}



Writing TP1/Parte-3/tp1_p3_c++.cpp


#### Compilacion

In [ ]:
!g++ -o exe TP1/Parte-3/tp1_p3_c++.cpp

#### Ejecucion

In [ ]:
./exe archivo1k.txt

# Código en JAVA




## Parte 1

### Creación archivo

In [ ]:
%%writefile TP1/Parte-1/tp1_p1_java.java
import java.io.IOException;
import java.util.ArrayList;
import java.util.Map;
import java.util.Set;

class ProcessTree
{
  static Map<Character, Set<Character>> processMap = Map.of (
      'A', Set.of('B', 'C', 'D'),
      'B', Set.of('E', 'F'),
      'F', Set.of('H', 'I'),
      'I', Set.of('J'),
      'D', Set.of('G')
  );

  static ArrayList<Process> processList = new ArrayList<>();

  static final int SLEEP_TIME = 100000;
  // Milliseconds

  public static void showInfo ( char processName )
  {
    ProcessHandle me = ProcessHandle.current();
    long pid  = me.pid();
    long ppid = me.parent().get().pid();
    System.out.println( "Process " + processName + ", PID: " + pid
        + ", PPID: " + ppid );
  }

  public static void createChilds ( char processName ) throws IOException,
      InterruptedException
  {
    if ( processMap.containsKey(processName) )
    {
      for ( char son : processMap.get(processName) )
      {
        ProcessBuilder builder = new ProcessBuilder("java",
            "TP1/Parte-1/tp1_p1_java.java", String.valueOf(son));
        builder.inheritIO();
        Process process = builder.start();
        processList.add(process);
      }
    }
    else
    {
      Thread.sleep(SLEEP_TIME);
    }
  }

  public static void waitChilds ( ) throws InterruptedException
  {
    for ( Process p : processList )
    {
      p.waitFor();
    }
  }

  public static void main ( String[] args ) throws IOException,
      InterruptedException
  {
    char root = 'A';
    char process = ( args.length > 0 ) ? args[0].charAt(0) : root;
    showInfo(process);
    createChilds(process);
    waitChilds();
  }
}

Writing TP1/Parte-1/tp1_p1_java.java


FileNotFoundError: [Errno 2] No such file or directory: 'TP1/Parte-1/tp1_p1_java.java'

### Ejecución en segundo plano

In [ ]:
!nohup java TP1/Parte-1/tp1_p1_java.java 1>salidaJava.txt 2>/dev/null &

### Lista de procesos en ejecución

In [ ]:
!ps -ef | grep tp1_p1_java.java

root        4771       1 47 22:01 ?        00:00:02 java TP1/Parte-1/tp1_p1_java.java
root        4793    4771 62 22:01 ?        00:00:01 java TP1/Parte-1/tp1_p1_java.java B
root        4795    4771 54 22:01 ?        00:00:01 java TP1/Parte-1/tp1_p1_java.java C
root        4798    4771 69 22:01 ?        00:00:02 java TP1/Parte-1/tp1_p1_java.java D
root        4865     225  0 22:01 ?        00:00:00 /bin/bash -c ps -ef | grep tp1_p1_java.java
root        4867    4865  0 22:01 ?        00:00:00 grep tp1_p1_java.java


### Árbol de procesos

In [ ]:
!pstree -pT 4771

java(4771)─┬─java(4793)─┬─java(4888)─┬─java(4948)───java(5004)
           │            │            └─java(4951)
           │            └─java(4891)
           ├─java(4795)
           └─java(4798)───java(4872)


### Salida del programa

In [ ]:
!cat salidaJava.txt

Process A, PID: 4771, PPID: 1
Process D, PID: 4798, PPID: 4771
Process B, PID: 4793, PPID: 4771
Process C, PID: 4795, PPID: 4771
Process G, PID: 4872, PPID: 4798
Process F, PID: 4888, PPID: 4793
Process E, PID: 4891, PPID: 4793
Process I, PID: 4948, PPID: 4888
Process H, PID: 4951, PPID: 4888
Process J, PID: 5004, PPID: 4948


## Parte 2

In [ ]:
%%writefile TP1/Parte-2/tp1_p2_java.java
import java.time.Duration;
import java.time.Instant;
import java.util.Arrays;
import java.util.Random;

public class Matrix
 {
    private static final int ORDER = 10000;
    private static final int THREADSNUM = 10;
    private static final int SCALAR = 1000000000;
    private static final int[][] mat = new int[ORDER][ORDER];
    private static final int[][] matConc = new int[ORDER][ORDER];
    private static final int[][] matSec = new int[ORDER][ORDER];
    private static final Random rand = new Random();

    public static void main(final String[] args)
    {

        initializeMatrix();

        System.out.println("scalar: " + SCALAR + " order: " + ORDER);

        if (ORDER <= 10)
        {
            printMatrix(mat);
        }

        final Instant startRs = Instant.now();
        secuentialScalarMultiplication();
        final Instant finishRs = Instant.now();

        System.out.println("Time of Sequential Matrix: " + Duration.between(startRs, finishRs).toMillis() + " ms");

        if (ORDER <= 10)
        {
            System.out.println("Matriz secuencial");
            printMatrix(matSec);
        }

        final Instant startRc = Instant.now();
        concurrentScalarMultiplication();
        final Instant finishRc = Instant.now();

        System.out.println("Time of Concurrent Matrix: " + Duration.between(startRc, finishRc).toMillis() + " ms");
        System.out.println("Using " + THREADSNUM + " threads");

        if (ORDER <= 10)
        {
            System.out.println("Matriz concurrente");
            printMatrix(matConc);
        }

        System.out.println("Are matrices matConc and matSec equal? " + Arrays.deepEquals(matConc, matSec));
    }

    public static void secuentialScalarMultiplication()
    {
        for (int i = 0; i < ORDER; i++)
        {
            for (int j = 0; j < ORDER; j++)
            {
                matSec[i][j] = mat[i][j] * SCALAR;
            }
        }
    }

    public static void concurrentScalarMultiplication()
    {
        final Thread[] threads = new Thread[THREADSNUM];

        final int rowsPerThread = ORDER / THREADSNUM;

        for (int i = 0; i < THREADSNUM; i++)
        {
            final int startRow = i * rowsPerThread;
            //Esta línea garantiza que el último hilo procese todas las filas restantes si el orden no es perfectamente divisible por THREADSNUM.
            //Garantiza que se procesen todas las filas, distribuyendo el trabajo de manera más uniforme.
            final int endRow = (i == THREADSNUM - 1) ? ORDER : (i + 1) * rowsPerThread;

            threads[i] = new Thread(() ->
             {
                for (int k = startRow; k < endRow; k++)
                {
                    for (int w = 0; w < ORDER; w++)
                    {
                        matConc[k][w] = mat[k][w] * SCALAR;
                    }
                }
            });

            threads[i].start();

        }

        for (final Thread thread : threads)
        {
            try
            {
                thread.join();
            }
            catch (final InterruptedException e)
            {
                e.printStackTrace();
                Thread.currentThread().interrupt();
            }
        }
    }

    public static void printMatrix(final int[][] mat)
    {
        for (final int[] row : mat)
        {
            for (final int col : row)
            {
                System.out.print(col + " ");
            }
            System.out.println();
        }

    }

    public static void initializeMatrix()
    {
        final int min = 0;
        final int max = 9;

        for (int i = 0; i < ORDER; i++)
        {
            for (int j = 0; j < ORDER; j++)
            {
                mat[i][j] = rand.nextInt(max - min + 1) + min;
            }
        }

    }
}


Overwriting TP1/Parte-2/tp1_p2_java.java


#### Ejecucion

In [ ]:
!java /content/TP1/Parte-2/tp1_p2_java.java

scalar: 1000000000 order: 10000
Time of Sequential Matrix: 96 ms
Time of Concurrent Matrix: 200 ms
Using 10 threads
Are matrices matConc and matSec equal? true


## Parte 3

#### Creacion archivo

In [ ]:
%%writefile TP1/Parte-3/tp1_p3_java.java
import java.util.NoSuchElementException;
import java.util.Scanner;

public class Parte3
{
    static final String SIDE_A = "LADO A";
    static final String SIDE_B = "LADO B";
    static final String BEGIN = SIDE_A;
    static final String FOX = "Zorro";
    static final String CHICKEN = "Pollo";
    static final String MAIZ = "Maiz";
    static final String FARMER = "Granjero";
    static final int OPTION_FARMER = 0;
    static final int OPTION_FOX = 1;
    static final int OPTION_CHICKEN = 2;
    static final int OPTION_MAIZ = 3;
    public static void main(String[] args)
    {
        InitialSide sideA = new InitialSide(SIDE_A);
        EndSide sideB = new EndSide(SIDE_B);

        sideA.setOtherSide(sideB);
        sideB.setOtherSide(sideA);

        sideA.start();
        sideB.start();
    }
    static public class SideThread extends Thread
    {
        protected String name;
        public StateLado state;
        public volatile boolean running = true;
        private String lastObject;

        public SideThread(String name)
        {
            this.name = name;
        }
        public synchronized void stopThread()
        {
            this.running = false;
        }

        public String getNameSide()
        {
            return this.name;
        }

        public synchronized boolean isRunning()
        {
            return running;
        }

        public String getLastObject()
        {
            return this.lastObject;
        }
        public void setLastObject(String object)
        {
            this.lastObject = object;
        }
    }
    static class InitialSide extends SideThread
    {
        private EndSide otherSide;

        public InitialSide(String name)
        {
            super(name);
            this.state = new StateLado(name,true);
        }
        public void setOtherSide(EndSide otherSide)
        {
            this.otherSide = otherSide;
        }

        public synchronized void run()
        {
            Scanner scanner = new Scanner(System.in);
            try{
                Mesagges.welcome();
                while(running)
                {
                    this.state.menuBack(scanner,otherSide);
                    if(!this.state.validate())
                    {
                        running = false;
                        otherSide.stopThread();
                        synchronized (otherSide)
                        {
                            otherSide.notify();
                        }
                        continue;
                    }
                    synchronized (otherSide)
                    {
                        otherSide.notify();
                    }
                    wait();
                    if(!running)
                        break;
                    if(this.getLastObject() != null)
                        this.state.refresh_states(this.getLastObject());
                    else
                        Mesagges.farmerAlone(this.state.site);

                    this.state.farmer_in_site = !this.state.farmer_in_site;
                    this.setLastObject(null);
                }

            }catch (InterruptedException e)
            {
                e.printStackTrace();
            }
            catch (NoSuchElementException e)
            {
                Mesagges.errorScanner();
            }finally
            {
                scanner.close();
                running = false;
                otherSide.stopThread();
            }
        }

    }

    static class EndSide extends SideThread {
        private InitialSide otherSide;

        public EndSide(String name) {
            super(name);
            this.state = new StateLado(name);
        }
        public void setOtherSide(InitialSide otherSide) {
            this.otherSide = otherSide;
        }

        public synchronized void run() {
            Scanner scanner = new Scanner(System.in);
            while (running) {
                try {
                    synchronized (this) {
                        wait();
                    }
                    if(!running)
                        break;
                    String last = this.getLastObject();
                    if(last != null && !last.isEmpty())
                    {
                        this.state.refresh_states(last);
                    }
                    else
                    {
                        Mesagges.farmerAlone(this.name);
                    }
                    this.state.farmer_in_site = !this.state.farmer_in_site;
                    if (this.state.checkWin())
                    {
                        Mesagges.winRiddle();
                        running = false;
                        otherSide.stopThread();
                        synchronized (otherSide)
                        {
                            otherSide.notify();
                        }
                        break;
                    }
                    state.menuBack(scanner,otherSide);
                    if(!this.state.validate())
                    {
                        running = false;
                        otherSide.stopThread();
                    }
                    this.setLastObject(null);
                    synchronized (otherSide) {
                        Mesagges.crossChange(name,otherSide.name);
                        otherSide.notify();
                    }
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }
        }
    }

    static class StateLado
    {
        private boolean fox_in_site;
        private boolean maiz_in_site;
        private boolean chicken_in_site;
        private boolean farmer_in_site;
        private String site;

        public StateLado(String site)
        {
            this.fox_in_site = false;
            this.maiz_in_site = false;
            this.chicken_in_site = false;
            this.farmer_in_site = false;
            this.site = site;
        }
        public StateLado(String site,boolean begin)
        {
            this.fox_in_site = begin;
            this.maiz_in_site = begin;
            this.chicken_in_site = begin;
            this.farmer_in_site = begin;
            this.site = site;
        }
        public boolean getFoxInSite(){
            return this.fox_in_site;
        }
        public void setFoxInSite(boolean fox_in_site){
            this.fox_in_site = fox_in_site;
        }
        public boolean getMaizInSite(){
            return this.maiz_in_site;
        }
        public void setMaizInSite(boolean maiz_in_site){
            this.maiz_in_site = maiz_in_site;
        }
        public boolean getChickenInSite(){
            return this.chicken_in_site;
        }
        public void setChickenInSite(boolean chicken_in_site){
            this.chicken_in_site = chicken_in_site;
        }
        public boolean getFarmerInSite(){
            return this.farmer_in_site;
        }
        public void setFarmerInSite(boolean farmer_in_site){
            this.farmer_in_site = farmer_in_site;
        }

        public synchronized boolean checkWin()
        {
            if(this.chicken_in_site && this.fox_in_site && this.maiz_in_site && this.farmer_in_site)
            {
                return true;
            }
            return false;
        }
        public synchronized void cross(String objeto, String otherSideString)
        {
            switch (objeto) {
                case FARMER:
                    break;
                case FOX:
                    this.fox_in_site = !this.fox_in_site;
                    break;
                case CHICKEN:
                    this.chicken_in_site = !this.chicken_in_site;
                    break;
                case MAIZ:
                    this.maiz_in_site = !this.maiz_in_site;
                    break;
            }
            this.farmer_in_site = !this.farmer_in_site;
        }
        public synchronized boolean validate()
        {
            if (this.fox_in_site && this.chicken_in_site && !this.farmer_in_site)
            {
                Mesagges.messageValidate(FOX, CHICKEN,this.site);
                return false;
            } else if (this.chicken_in_site && this.maiz_in_site && !this.farmer_in_site)
            {
                Mesagges.messageValidate(CHICKEN, MAIZ, this.site);
                return false;
            }
            return true;
        }
        public void menuBack(Scanner scanner,SideThread otherSide)
        {
            while (true)
            {
                System.out.println("======================== MENÚ LADO "+ site +" ========================");
                System.out.println("Por favor, ingrese cómo desea que el granjero viaje:");
                System.out.println(this.farmer_in_site ? "0. Granjero solo" : "");
                System.out.println(this.fox_in_site ? "1. Zorro" : "");
                System.out.println(this.chicken_in_site ? "2. Chicken" : "");
                System.out.println(this.maiz_in_site ? "3. Maiz" : "");
                System.out.print("¿Cruzar con?: ");
                int option = scanner.nextInt();
                if(processOption(option,otherSide) == true)
                    break;
            }
        }

        public synchronized boolean processOption(int option, SideThread otherSide)
        {
            switch (option)
            {
                case OPTION_FARMER:
                    cross(FARMER, otherSide.getNameSide());
                    break;
                case OPTION_FOX:
                    if (this.fox_in_site)
                    {
                        cross(FOX, otherSide.getNameSide());
                        otherSide.setLastObject(FOX);
                    } else {
                        Mesagges.messageOtherSide(FOX);
                        return false;
                    }
                    break;
                case OPTION_CHICKEN:
                    if (this.chicken_in_site)
                    {
                        cross(CHICKEN, otherSide.getNameSide());
                        otherSide.setLastObject(CHICKEN);
                    } else
                    {
                        Mesagges.messageOtherSide(CHICKEN);
                        return false;
                    }
                    break;
                case OPTION_MAIZ:
                    if (this.maiz_in_site)
                    {
                        cross(MAIZ, otherSide.getNameSide());
                        otherSide.setLastObject(MAIZ);
                    } else
                    {
                        Mesagges.messageOtherSide(MAIZ);
                        return false;
                    }
                    break;
                default:
                    Mesagges.messageError();
                    return false;
            }
            return true;
        }
        public void refresh_states(String option)
        {
            if(option != null)
            {
                switch (option)
                {
                    case FOX:
                        this.fox_in_site = !this.fox_in_site;
                        Mesagges.messageCross(FOX, site);
                        break;
                    case CHICKEN:
                        this.chicken_in_site = !this.chicken_in_site;
                        Mesagges.messageCross(CHICKEN, site);
                        break;
                    case MAIZ:
                        this.maiz_in_site = !this.maiz_in_site;
                        Mesagges.messageCross(MAIZ, site);
                        break;
                    default:
                        break;
                }
            }

        }

    }
    public static class Mesagges
    {
        public static void welcome()
        {
            System.out.println("Bienvenido al problema del Zorro, el Pollo y el Maiz.");
        }
        public static void messageError()
        {
            System.out.println("Se ha producido un error. Inténtalo de nuevo.");
        }
        public static void invalidOption()
        {
            System.out.println("Opción inválida.");
        }
        public static void messageOtherSide(String object)
        {
            System.out.println("El " + object +" ya está del otro lado.");
        }
        public static void messageValidate(String object, String otherObject, String side)
        {
            System.out.println("¡La "+ object +" se comió el "+ otherObject +"! En el "+side);
            System.out.println("Fin del programa");
        }
        public static void messageCross(String object, String side)
        {
            if(object==null)
                System.out.println(FARMER + " se encuentran en "+ side);
            else
                System.out.println(FARMER + " y " + object + " se encuentran en "+side);
        }
        public static void crossChange(String side, String otherSide)
        {
            System.out.println("Cruzando de " + side + " a " + otherSide);
        }
        public static void farmerAlone(String side)
        {
            System.out.println("El granjero llego a " + side + " solo." );
        }

        public static void errorScanner()
        {
            System.out.println("Se ha producido un error al leer la entrada. Inténtalo de nuevo.");
        }
        public static void winRiddle()
        {
            System.out.println("Todos se encuentran en lado B");
            System.out.println("Fin del programa");
        }
    }
}


Overwriting TP1/Parte-3/tp1_p3_java.java


#### Ejecucion

In [ ]:
!java /content/TP1/Parte-3/tp1_p3_java.java

Bienvenido al problema del Zorro, el Pollo y el Maiz.
======================== MENÚ LADO LADO A ========================
Por favor, ingrese cómo desea que el granjero viaje:
0. Granjero solo
1. Zorro
2. Chicken
3. Maiz
¿Cruzar con?: 2
Granjero y Pollo se encuentran en LADO B
======================== MENÚ LADO LADO B ========================
Por favor, ingrese cómo desea que el granjero viaje:
0. Granjero solo

2. Chicken

¿Cruzar con?: 0
Cruzando de LADO B a LADO A
El granjero llego a LADO A solo.
======================== MENÚ LADO LADO A ========================
Por favor, ingrese cómo desea que el granjero viaje:
0. Granjero solo
1. Zorro

3. Maiz
¿Cruzar con?: 1
Granjero y Zorro se encuentran en LADO B
======================== MENÚ LADO LADO B ========================
Por favor, ingrese cómo desea que el granjero viaje:
0. Granjero solo
1. Zorro
2. Chicken

¿Cruzar con?: 2
Cruzando de LADO B a LADO A
Granjero y Pollo se encuentran en LADO A
======================== MENÚ LADO LADO A 

# Codigo en Rust




## Parte 1

### Dependencias
Instalamos el compilador de rust, el administrador de proyectos cargo, y la libreria a utilizar

In [ ]:
!sudo apt-get install rustc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libstd-rust-1.75 libstd-rust-dev
Suggested packages:
  cargo llvm-17 lld-17 clang-17
The following NEW packages will be installed:
  libstd-rust-1.75 libstd-rust-dev rustc
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 91.4 MB of archives.
After this operation, 369 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libstd-rust-1.75 amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [46.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libstd-rust-dev amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [41.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 rustc amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [3,404 kB]
Fetched 91.4 MB in 4s (23.2 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-lik

In [ ]:
!sudo apt-get install cargo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  cargo-doc
The following NEW packages will be installed:
  cargo
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 6,700 kB of archives.
After this operation, 23.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 cargo amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [6,700 kB]
Fetched 6,700 kB in 0s (17.2 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pac

In [ ]:
!cargo --version && rustc --version

cargo 1.75.0
rustc 1.75.0 (82e1608df 2023-12-21) (built from a source tarball)


### Crear el archivo

creamos un nuevo proyecto en donde se va a ejecutar el codigo

In [ ]:
!cargo new /content/TP1/Parte-1/ej1_tp1_rust

     Created binary (application) `/content/TP1/Parte-1/ej1_tp1_rust` package


agregamos la libreria externa necesaria para el código

In [ ]:
!cd /content/TP1/Parte-1/ej1_tp1_rust && cargo add nix --features process

/bin/bash: line 1: cd: /content/TP1/Parte-1/ej1_tp1_rust: No such file or directory


creamos el archivo

In [ ]:
%%writefile /content/TP1/Parte-1/ej1_tp1_rust/src/main.rs
use nix::sys::wait::{wait};
use nix::unistd::{fork, ForkResult};
use std::collections::HashMap;
use std::io::{Error, ErrorKind};
use std::process::id;
use std::{os::unix::process::parent_id, process::Command};

fn main()
{
    let process_tree: HashMap<char, Vec<char>> = HashMap::from([
        ('A', vec!['B', 'C', 'D']),
        ('B', vec!['E', 'F']),
        ('D', vec!['G']),
        ('F', vec!['H', 'I']),
        ('I', vec!['J']),
    ]);

    let seconds = 10;

    if let Err(e) = create_fork(&process_tree, &'A', &seconds)
    {
        eprintln!("Error: {}", e);
        std::process::exit(1);
    }
}

fn create_fork(
    process_tree: &HashMap<char, Vec<char>>,
    initial_node: &char,
    seconds: &u32,
) -> Result<(), Error>
{
    println!("Node {} id {} pid {}", initial_node, id(), parent_id());
    sleep_seconds(&1);

    match process_tree.get(&initial_node)
     {
        Some(children) =>
        {
            for &next_child in children
            {
                match unsafe
                {
                  fork()
                }
                {
                    Ok(ForkResult::Parent { .. }) => {}
                    Ok(ForkResult::Child) =>
                    {
                        if let Err(e) = create_fork(process_tree, &next_child, seconds)
                        {
                            eprintln!("Error in child node: {}", e);
                            std::process::exit(1)
                        }
                        std::process::exit(0)
                    }
                    Err(e) => return Err(Error::new(ErrorKind::Other, format!("Error in fork: {}", e))),
                }
            }
            for _ in 0..children.len()
            {
                let _ = wait();
            }
        }
        None => sleep_seconds(seconds),
    }

    println!("Ending node: {}", initial_node);
    Ok(())
}

fn sleep_seconds(seconds: &u32)
{
    Command::new("sleep")
        .arg(&seconds.to_string())
        .output()
        .expect("failure to execute process");
}

Overwriting /content/TP1/Parte-1/ej1_tp1_rust/src/main.rs


### Compilacion

In [ ]:
!cd /content/TP1/Parte-1/ej1_tp1_rust && cargo build

  Downloaded bitflags v2.5.0
  Downloaded cfg_aliases v0.1.1
  Downloaded cfg-if v1.0.0
  Downloaded nix v0.28.0
  Downloaded libc v0.2.153
   Compiling libc v0.2.153
   Compiling cfg_aliases v0.1.1
   Compiling nix v0.28.0
   Compiling cfg-if v1.0.0
   Compiling bitflags v2.5.0
   Compiling ej1_tp1_rust v0.1.0 (/content/TP1/Parte-1/ej1_tp1_rust)
    Finished dev [unoptimized + debuginfo] target(s) in 7.63s


### Ejecución en segundo plano

In [ ]:
!nohup /content/TP1/Parte-1/ej1_tp1_rust/target/debug/ej1_tp1_rust 1>/content/TP1/Parte-1/salidaRust 2>/dev/null &

### visualización de arbol de procesos

In [ ]:
!pstree -p | grep ej1_tp1_rust

docker-init(1)-+-ej1_tp1_rust(2799)-+-ej1_tp1_rust(2805)-+-ej1_tp1_rust(2816)---sleep(2819)
               |                    |                    `-ej1_tp1_rust(2817)---sleep(2821)
               |                    |-ej1_tp1_rust(2806)---sleep(2820)
               |                    `-ej1_tp1_rust(2808)---ej1_tp1_rust(2815)---sleep(2818)


visualizacion output

In [ ]:
!cat /content/TP1/Parte-1/salidaRust

Node A id 2799 pid 1
Node B id 2805 pid 2799
Node D id 2808 pid 2799
Node C id 2806 pid 2799
Node G id 2815 pid 2808
Node E id 2816 pid 2805
Node F id 2817 pid 2805
Node I id 2831 pid 2817
Node H id 2830 pid 2817
Node J id 2840 pid 2831
Ending node: C
Ending node: E
Ending node: G
Ending node: D
Ending node: H
Ending node: J
Ending node: I
Ending node: F
Ending node: B
Ending node: A


## Parte 2

### Crear el archivo

In [ ]:
!cargo new /content/TP1/Parte-2/ej2_p2_rust

     Created binary (application) `/content/TP1/Parte-2/ej2_p2_rust` package


In [ ]:
!cd /content/TP1/Parte-2/ej2_p2_rust && cargo add nix --features process

    Updating crates.io index
      Adding nix v0.28.0 to dependencies.
             Features:
             + process
             34 deactivated features
    Updating crates.io index


Creamos el archivo

In [ ]:
%%writefile /content/TP1/Parte-2/ej2_p2_rust/src/main.rs
  use std::thread;
  use std::time::Instant;
  use rand::Rng;

  #[derive(Debug)]
  struct Matrix
  {
      matrix: Vec<Vec<i32>>,
      order: usize,
  }

  impl Matrix
  {
      pub fn new(order: usize) -> Self
      {
          let matrix = vec![vec![0; order]; order];
          Matrix
          {
            matrix, order
          }
      }

      pub fn fill_matrix(&mut self)
      {
          let n = self.order;
          for i in 0..n
          {
              for j in 0..n
              {
                  self.matrix[i][j] = rand::thread_rng().gen_range(0..10);
              }
          }
      }

      pub fn multiply_sequential(&self, scalar: i32) -> Self
      {
          let mut result = self.matrix.clone();
          for i in 0..self.order
          {
              for j in 0..self.order
              {
                  result[i][j] *= scalar;
              }
          }
          Matrix
          {
              matrix: result,
              order: self.order,
          }
      }

      pub fn multiply_concurrent(&self, scalar: i32, num_threads: usize) -> Self
      {
        let rows_per_thread = self.order / num_threads;
        let mut handles = Vec::with_capacity(num_threads);

        let mut result_matrix = Vec::with_capacity(self.order * self.order);
        for i in 0..num_threads
        {
            let start_row = i * rows_per_thread;
            let mut sub_matrix = self.matrix[start_row..(start_row + rows_per_thread)].to_vec();

            let handle = thread::spawn(move ||
            {
                for row in &mut sub_matrix
                {
                    for elem in row
                    {
                        *elem *= scalar;
                    }
                }
                sub_matrix
            });

            handles.push(handle);
        }

        for handle in handles
        {
            let sub_matrix = handle.join().unwrap();
            result_matrix.extend(sub_matrix);
        }

        Matrix
        {
            matrix: result_matrix,
            order: self.order,
        }
    }

  }

  fn main()
  {
      let n = 1000;
      let scalar = 2;
      let threads = 100;

      let mut original_matrix = Matrix::new(n);
      original_matrix.fill_matrix();

      let sequential_start = Instant::now();
      original_matrix.multiply_sequential(scalar);
      let sequential_duration = sequential_start.elapsed();

      let concurrent_start = Instant::now();
      original_matrix.multiply_concurrent(scalar, threads);
      let concurrent_duration = concurrent_start.elapsed();

      println!("Elapsed time for sequential multiplication: {:?} ms", sequential_duration.as_millis());
      println!("Elapsed time for concurrent multiplication: {:?} ms", concurrent_duration.as_millis());
  }

Overwriting /content/TP1/Parte-2/ej2_p2_rust/src/main.rs


### Compilacion

In [ ]:
!cd /content/TP1/Parte-2/ej2_p2_rust && cargo build

   Compiling ej2_p2_rust v0.1.0 (/content/TP1/Parte-2/ej2_p2_rust)
    Finished dev [unoptimized + debuginfo] target(s) in 0.63s


In [ ]:
!cargo run --quiet --manifest-path TP1/Parte-2/ej2_p2_rust/Cargo.toml

Elapsed time for sequential multiplication: 34259 microseconds
Elapsed time for concurrent multiplication: 20367 microseconds


# Código en GO


## Instalar paquete Goland

In [ ]:
!apt-get install -y golang-go

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  golang-1.18-go golang-1.18-src golang-src
Suggested packages:
  bzr | brz mercurial subversion
The following NEW packages will be installed:
  golang-1.18-go golang-1.18-src golang-go golang-src
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 82.3 MB of archives.
After this operation, 436 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-src all 1.18.1-1ubuntu1.1 [16.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-go amd64 1.18.1-1ubuntu1.1 [66.0 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-src all 2:1.18~0ubuntu2 [4,438 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-go amd64 2:1.18~0ubuntu2 [41.8 kB]
Fetched 82.3 MB in 5s (17.8 MB/s)
Selecting previously unselected pa

## Parte 1

### Crear Archivo

In [ ]:
%%writefile TP1/Parte-1/Parte1_go.go
package main

import (
	"fmt"
	"os"
	"os/exec"
	"sync"
	"time"
)

const RAIZ = 'A'
const ARGUMENTOS = 1
const CHILDLESS = 0
const SECONDS_WAIT = 10

type ProcessPrinter struct {
	mu sync.Mutex
}

var myDictionary = map[rune][]rune{
	'A': {'B', 'C', 'D'},
	'B': {'E', 'F'},
	'C': {},
	'D': {'G'},
	'E': {},
	'F': {'H', 'I'},
	'G': {},
	'H': {},
	'I': {'J'},
	'J': {},
}

var printer = ProcessPrinter{}

func main() {
	var wg sync.WaitGroup
	var process rune

	if len(os.Args) == ARGUMENTOS {
		process = rune(RAIZ)
	} else {
		process = rune(os.Args[1][0])
	}

	printer.showProcess(string(process))
	children, ok := myDictionary[process]

	if !ok {
		fmt.Println("Clave no encontrada en el diccionario")
		return
	}

	if len(children) > CHILDLESS {
		wg.Add(len(children))
		for _, letter := range children {
		   createChildProcess(letter, &wg)
		}
	}
	time.Sleep(SECONDS_WAIT * time.Second)
	wg.Wait()
}
func createChildProcess(name rune, wg *sync.WaitGroup) {
	defer wg.Done()

	cmd := exec.Command(os.Args[0], string(name))
	cmd.Stdout = os.Stdout
	cmd.Stderr = os.Stderr

	if err := cmd.Start(); err != nil {
		fmt.Println("Error al ejecutar el proceso hijo:", err)
	}
}
func (pp *ProcessPrinter) showProcess(process string) {
	pp.mu.Lock()
	defer pp.mu.Unlock()
	fmt.Println("Proceso", process, "pid:", os.Getpid(), "ppid:", os.Getppid())
}

Writing TP1/Parte-1/Parte1_go.go


### Compilacion



In [ ]:
!go run TP1/Parte-1/Parte1_go.go

Proceso A pid: 10183 ppid: 10144
Proceso B pid: 10187 ppid: 10183
Proceso D pid: 10188 ppid: 10183
Proceso C pid: 10198 ppid: 10183
Proceso F pid: 10199 ppid: 10187
Proceso E pid: 10200 ppid: 10187
Proceso G pid: 10208 ppid: 10188
Proceso I pid: 10212 ppid: 10199
Proceso H pid: 10213 ppid: 10199
Proceso J pid: 10226 ppid: 10212


### Ejecución en segundo plano

In [ ]:
!nohup go run TP1/Parte-1/Parte1_go.go 1>salidaGo.txt 2>/dev/null &

### Ver arbol de procesos

In [ ]:
!ps -axf | grep Parte1_go

   1217 ?        S      0:00  |       \_ /bin/bash -c ps -axf | grep Parte1_go
   1219 ?        S      0:00  |           \_ grep Parte1_go
   1139 ?        Sl     0:00 go run TP1/Parte-1/Parte1_go.go
   1169 ?        Sl     0:00  \_ /tmp/go-build1789597745/b001/exe/Parte1_go
   1174 ?        Sl     0:00      \_ /tmp/go-build1789597745/b001/exe/Parte1_go B
   1181 ?        Sl     0:00      |   \_ /tmp/go-build1789597745/b001/exe/Parte1_go E
   1188 ?        Sl     0:00      |   \_ /tmp/go-build1789597745/b001/exe/Parte1_go F
   1197 ?        Sl     0:00      |       \_ /tmp/go-build1789597745/b001/exe/Parte1_go H
   1201 ?        Sl     0:00      |       \_ /tmp/go-build1789597745/b001/exe/Parte1_go I
   1209 ?        Sl     0:00      |           \_ /tmp/go-build1789597745/b001/exe/Parte1_go J
   1175 ?        Sl     0:00      \_ /tmp/go-build1789597745/b001/exe/Parte1_go C
   1179 ?        Sl     0:00      \_ /tmp/go-build1789597745/b001/exe/Parte1_go D
   1192 ?        Sl     0:00    


### Arbol de procesos

In [ ]:
!pstree -pT 1169

Parte1_go(1169)─┬─Parte1_go(1174)─┬─Parte1_go(1181)
                │                 └─Parte1_go(1188)─┬─Parte1_go(1197)
                │                                   └─Parte1_go(1201)───Parte1_go(1209)
                ├─Parte1_go(1175)
                └─Parte1_go(1179)───Parte1_go(1192)


### Salida del programa

In [ ]:
!cat salidaGo.txt


Proceso A pid: 1169 ppid: 1139
Proceso B pid: 1174 ppid: 1169
Proceso C pid: 1175 ppid: 1169
Proceso D pid: 1179 ppid: 1169
Proceso E pid: 1181 ppid: 1174
Proceso F pid: 1188 ppid: 1174
Proceso G pid: 1192 ppid: 1179
Proceso H pid: 1197 ppid: 1188
Proceso I pid: 1201 ppid: 1188
Proceso J pid: 1209 ppid: 1201


## Parte 2

#### Crear Archivo

In [ ]:
%%writefile TP1/Parte-2/Parte2_go.go
package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

const SIZE int = 9000
const THREADS int = 100
const ESCALAR int = 3

var matrixA [SIZE][SIZE]int
var matrixResSequ [SIZE][SIZE]int
var matrixResConc [SIZE][SIZE]int

func printMatrix(matrix *[SIZE][SIZE]int) {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			fmt.Print("|")
			if matrix[i][j] < 10 {
				fmt.Print(" ")
			}
			if matrix[i][j] < 100 {
				fmt.Print(" ")
			}
			fmt.Print(matrix[i][j])
			fmt.Print("|")
		}
		fmt.Println()
	}
}

func initMatrix(m *[SIZE][SIZE]int) {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			m[i][j] = rand.Intn(100)
		}
	}
}

func multSeqMatrix() {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			matrixResSequ[i][j] = matrixA[i][j] * ESCALAR
		}
	}
}
func multConcMatrix() {
	var wg sync.WaitGroup
	wg.Add(THREADS)

	blockSize := SIZE / THREADS

	for i := 0; i < THREADS; i++ {
		go multRows(i*blockSize, (i+1)*blockSize, &wg)
	}

	wg.Wait()
}

func multRows(startRow int, endRow int, wg *sync.WaitGroup) {
	defer wg.Done()
	for r := startRow; r < endRow; r++ {
		for c := 0; c < SIZE; c++ {
			matrixResConc[r][c] = matrixA[r][c] * ESCALAR
		}
	}
}

func compareRes() bool {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			if matrixResConc[i][j] != matrixResSequ[i][j] {
				fmt.Println(matrixResConc[i][j], "!=", matrixResSequ[i][j], "i", i, "j", j)
				return false
			}
		}
	}
	return true
}
func printAllMatrix() {
	fmt.Println("Matrix A:")
	printMatrix(&matrixA)

	fmt.Println("\nRes Seq:")
	printMatrix(&matrixResSequ)

	fmt.Println("\nRes Conc:")
	printMatrix(&matrixResConc)
}

func main() {
	fmt.Println("matrix size: ", SIZE)
	fmt.Println("Threads: ", THREADS)
	initMatrix(&matrixA)

	startSeq := time.Now()
	multSeqMatrix()
	endSeq := time.Since(startSeq).Milliseconds()

	startConc := time.Now()
	multConcMatrix()
	endConc := time.Since(startConc).Milliseconds()

	// printAllMatrix()

	fmt.Println("\nSECUENCIAL == CONCURRENTE:", compareRes())

	fmt.Println("SECUENCIAL: ", endSeq)
	fmt.Println("CONCURRENTE: ", endConc)
}


Writing TP1/Parte-2/Parte2_go.go


### Compilacion

In [ ]:
!go run TP1/Parte-2/Parte2_go.go

matrix size:  10000
Threads:  100

SECUENCIAL == CONCURRENTE: true
SECUENCIAL:  579
CONCURRENTE:  440


# Codigo en Python

## Parte 1

In [ ]:
%%writefile TP1/Parte-1/tp1_p1_python.py
import os
import time

TIME_SLEEP=20000


def imprimirProceso(proceso):
    pid = os.getpid()
    ppid = os.getppid()
    print(f"Proceso {proceso}: PID = {pid}, PID del Padre = {ppid}")

def crearProcesos(procesos, padre):
    hijos = procesos.get(padre, [])
    imprimirProceso(padre)
    for hijo in hijos:
        pid = os.fork()
        if pid < 0:
            print("Error pid del proceso")
        if pid == 0:  # es hijo
            crearProcesos(procesos, hijo)
            time.sleep(TIME_SLEEP)
            os._exit(0)
    for hijo in hijos:
        os.wait()


print("Comienzo a crear los procesos...\n")

procesos = {
    "A": ["B", "C", "D"],
    "B": ["E", "F"],
    "D": ["G"],
    "F": ["H", "I"],
    "I": ["J"]
}

crearProcesos(procesos, list(procesos.keys())[0])

print("\nCreación de procesos finalizada")

Writing TP1/Parte-1/tp1_p1_python.py


### Ver si se creo

In [ ]:
!ls -laR | grep tp1_p1_python.py

-rw-r--r-- 1 root root  828 Apr 11 18:29 tp1_p1_python.py


### Ejecución en segundo plano




In [ ]:
!nohup python3 TP1/Parte-1/tp1_p1_python.py &

nohup: appending output to 'nohup.out'


### Visualización de los procesos que están corriendo

In [ ]:
!ps -aufx

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root          22  0.0  0.0   5808  1000 ?        Ss   21:45   0:00 tail -n +0 -F /root/.config/Googl
root           1  0.0  0.0   1076     8 ?        Ss   21:45   0:00 /sbin/docker-init -- /datalab/run
root           6  0.5  0.3 886052 47516 ?        Rl   21:45   0:01 /tools/node/bin/node /datalab/web
root          16  0.0  0.0   7376  3428 ?        S    21:45   0:00  \_ /bin/bash -e /usr/local/colab
root        1057  0.0  0.0   5776  1000 ?        S    21:49   0:00  |   \_ sleep 1
root          61  3.3  0.0      0     0 ?        Z    21:45   0:07  \_ [python3] <defunct>
root          62  0.3  0.3  66516 51504 ?        S    21:45   0:00  \_ python3 /usr/local/bin/colab-
root          84  1.8  0.8 358404 112592 ?       Rl   21:45   0:03  \_ /usr/bin/python3 /usr/local/b
root         225  1.5  0.7 1109960 96668 ?       Ssl  21:45   0:02  |   \_ /usr/bin/python3 -m colab
root        1058  0.0  0.0  10076  1596 ?   

In [ ]:
!ps aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0   1076     8 ?        Ss   21:45   0:00 /sbin/docker-init -- /datalab/run
root           6  0.5  0.3 886052 47308 ?        Rl   21:45   0:01 /tools/node/bin/node /datalab/web
root          16  0.0  0.0   7376  3428 ?        S    21:45   0:00 /bin/bash -e /usr/local/colab/bin
root          18  0.0  0.0   7376  1864 ?        S    21:45   0:00 /bin/bash -e /datalab/run.sh
root          20  0.0  0.0 1235948 12892 ?       Sl   21:45   0:00 /usr/colab/bin/kernel_manager_pro
root          22  0.0  0.0   5808  1000 ?        Ss   21:45   0:00 tail -n +0 -F /root/.config/Googl
root          61  3.2  0.0      0     0 ?        Z    21:45   0:07 [python3] <defunct>
root          62  0.3  0.3  66516 51504 ?        S    21:45   0:00 python3 /usr/local/bin/colab-file
root          84  1.7  0.8 358404 112592 ?       Rl   21:45   0:03 /usr/bin/python3 /usr/local/bin/j
root          85  0.0  0.0 1227460 

###Visualización de procesos

In [ ]:
!ps -aux | grep TP1/Parte-1/tp1_p1_python.py

root        1126  0.0  0.0   6624  2208 ?        S    21:49   0:00 grep TP1/Parte-1/tp1_p1_python.py


### Visualización árbol de procesos

In [ ]:
!pstree -p 1000

python3(1000)─┬─python3(1001)─┬─python3(1004)
              │               └─python3(1005)─┬─python3(1007)
              │                               └─python3(1008)───python3(1009)
              ├─python3(1002)
              └─python3(1003)───python3(1006)


##Parte 2

#### Crear Archivo

In [ ]:
%%writefile TP1/Parte-2/tp1_p2_python.py
import numpy as np
import threading
import time

LIMITE_SUPERIOR = 9
LIMITE_INFERIOR = 0
TAM_MATRIZ = 1000
CANT_HILOS = 2
ESCALAR = 5
TIME_MILISECONDS = 1000

def create_matriz():
    return np.random.randint(LIMITE_INFERIOR,LIMITE_SUPERIOR, size=(TAM_MATRIZ, TAM_MATRIZ))

def compare_equals_matrices(matriz, matrizB):
    return np.array_equal(matriz,matrizB)

def inicializar_matriz():
    result_matriz = []
    for i in range(TAM_MATRIZ):
        row = []
        for j in range(TAM_MATRIZ):
            row.append(0)
        result_matriz.append(row)
    return result_matriz

def product_matrizXscalar_secuencial():
    for i in range(TAM_MATRIZ):
        for j in range(TAM_MATRIZ):
            matrizCS[i][j] = matriz_A[i][j] * ESCALAR

    return matrizCS

def product_matrizXscalar(start, end):
    for i in range(start,end):
        for j in range(TAM_MATRIZ):
            matrizCC[i][j] = matriz_A[i][j] * ESCALAR

def product_matrizXscalar_concurrente():
    threads = []
    blockSize = TAM_MATRIZ // CANT_HILOS
    for i in range(CANT_HILOS):
        thread = threading.Thread(target=product_matrizXscalar, args=(i*blockSize,(i+1)*blockSize,))
        thread.start()
        threads.append(thread)

    for j in threads:
        j.join()

matrizCC = inicializar_matriz()
matrizCS = inicializar_matriz()
matriz_A = create_matriz()

print(f"La matriz cuadrada es {TAM_MATRIZ} x {TAM_MATRIZ}")
print(f"Hilos ejecutandose concurrentemente: {CANT_HILOS}")

calculate_time_concurrente = time.time()
product_matrizXscalar_concurrente()
elapsed_time_2_ms = (time.time() - calculate_time_concurrente) * TIME_MILISECONDS

calculate_time_secuencial = time.time()
product_matrizXscalar_secuencial()
elapsed_time_1_ms = (time.time() - calculate_time_secuencial) * TIME_MILISECONDS

print(f"La aplicacion SECUENCIAL tardó {elapsed_time_1_ms} milisegundos en ejecutarse.")
print(f"La aplicacion CONCURRENTE tardó {elapsed_time_2_ms} milisegundos en ejecutarse.")

equals = compare_equals_matrices(matrizCC,matrizCS)

if equals:
    print("Las matrices son iguales")
else:
    print("Las matrices no son iguales")

Overwriting TP1/Parte-2/tp1_p2_python.py


### Compilacion

In [ ]:
!python3 TP1/Parte-2/tp1_p2_python.py

La matriz cuadrada es 1000 x 1000
Hilos ejecutandose concurrentemente: 2
La aplicacion SECUENCIAL tardó 425.13155937194824 milisegundos en ejecutarse.
La aplicacion CONCURRENTE tardó 447.1428394317627 milisegundos en ejecutarse.
Las matrices son iguales


## Parte 3

### Creación Archivo

In [ ]:
%%writefile TP1/Parte-3/tp1_p3_python.py
from collections import Counter
from multiprocessing import Pipe
import os
import sys

SUCURSAL_B = 1
SUCURSAL_C = 2
CHILD = 0
CANT_EMPATE = 3


def escribir_registros(archivo):
    datos = """SUCURSAL,FECHA,PRODUCTO,MONTO
1,01-05-2024,Producto A,100.00
1,01-05-2024,Producto B,150.00
1,01-05-2024,Producto D,60.00
1,02-05-2024,Producto C,80.00
2,02-05-2024,Producto D,130.00
1,03-05-2024,Producto D,75.00
2,04-05-2024,Producto A,85.00
1,05-05-2024,Producto C,160.00
2,06-05-2024,Producto C,95.00
1,07-05-2024,Producto D,180.00
1,08-05-2024,Producto B,120.00
2,09-05-2024,Producto C,110.00
1,10-05-2024,Producto B,200.00
1,10-05-2024,Producto E,150.00
1,11-05-2024,Producto A,140.00
2,12-05-2024,Producto B,90.00
1,13-05-2024,Producto C,170.00"""

    with open(archivo, 'w') as f:
        f.write(datos)


def leer_registros(archivo):
    registros = []
    with open(archivo, 'r') as file:
        next(file)  # Ignorar la primera línea (encabezados)
        for linea in file:
            sucursal, fecha, producto, monto = linea.strip().split(',')
            registros.append({
                'SUCURSAL': int(sucursal),
                'FECHA': fecha,
                'PRODUCTO': producto,
                'MONTO': float(monto)
            })
    return registros


def procesar_datos(registros):
    sucursal = registros[0]['SUCURSAL']
    total_monto = 0
    productos_cant_ventas = Counter()
    fechas_cant_ventas = Counter()
    montos_por_fecha = Counter()

    for registro in registros:
        monto = registro['MONTO']
        producto = registro['PRODUCTO']
        fecha = registro['FECHA']

        total_monto += monto
        productos_cant_ventas[producto] += 1
        fechas_cant_ventas[fecha] += 1
        montos_por_fecha[fecha] += monto

    producto_mas_vendido = max(productos_cant_ventas,
                               key=productos_cant_ventas.get)

    max_cant_ventas = max(fechas_cant_ventas.values())
    fechas_max_cant_ventas = [
        fecha for fecha, cantidad in fechas_cant_ventas.items()
        if cantidad == max_cant_ventas
    ]
    # Limitar las fechas a las primeras 3 en caso de empate
    fechas_max_cant_ventas = fechas_max_cant_ventas[:CANT_EMPATE]

    max_monto_fecha = max(montos_por_fecha.values())
    fechas_max_monto = [
        fecha for fecha, monto in montos_por_fecha.items()
        if monto == max_monto_fecha
    ]
    fechas_max_monto = fechas_max_monto[:CANT_EMPATE]

    resultados = {
        'Número de sucursal': sucursal,
        'Monto total': total_monto,
        'Producto más vendido': producto_mas_vendido,
        'Fecha/s con mayor cantidad de ventas': fechas_max_cant_ventas,
        'Mayor cantidad de ventas en una fecha': max_cant_ventas,
        'Fecha/s con mayor monto vendido': fechas_max_monto,
        'Mayor monto vendido en una fecha': max_monto_fecha
    }
    return resultados


def imprimir_resultados(resultados):
    for clave, valor in resultados.items():
        print(f"{clave}: {valor}")
    print("\n")


if __name__ == '__main__':
    archivo = 'registros_ventas.txt'
    pipe_ab, pipe_ba = Pipe()  # Para la comunicación entre A y B
    pipe_ac, pipe_ca = Pipe()  # Para la comunicación entre A y C

    escribir_registros(archivo)
    registros = leer_registros(archivo)
    registrosB = []
    registrosC = []

    for registro in registros:
        if registro['SUCURSAL'] == SUCURSAL_B:
            registrosB.append(registro)
        elif registro['SUCURSAL'] == SUCURSAL_C:
            registrosC.append(registro)

    pid_b = os.fork()
    if pid_b < 0:
        sys.exit("Error al crear el proceso hijo B.")

    if pid_b == CHILD:  # Proceso hijo B
        pipe_ab.close()
        registrosB = pipe_ba.recv()
        resultadosB = procesar_datos(registrosB)
        pipe_ba.send(resultadosB)
        pipe_ba.close()
        os._exit(0)
    else:  # Proceso padre A
        pipe_ba.close()
        pipe_ab.send(registrosB)
        resultadosB = pipe_ab.recv()
        pipe_ab.close()
        os.wait()

    pid_c = os.fork()
    if pid_c < 0:
        sys.exit("Error al crear el proceso hijo C.")

    if pid_c == CHILD:  # Proceso hijo C
        pipe_ac.close()
        registrosC = pipe_ca.recv()
        resultadosC = procesar_datos(registrosC)
        pipe_ca.send(resultadosC)
        pipe_ca.close()
        os._exit(0)
    else:  # Proceso padre A
        pipe_ca.close()
        pipe_ac.send(registrosC)
        resultadosC = pipe_ac.recv()
        pipe_ac.close()
        os.wait()

imprimir_resultados(resultadosB)
imprimir_resultados(resultadosC)


Overwriting TP1/Parte-3/tp1_p3_python.py


### Ejecución

In [ ]:
!python3 TP1/Parte-3/tp1_p3_python.py

Número de sucursal: 1
Monto total: 1585.0
Producto más vendido: Producto B
Fecha/s con mayor cantidad de ventas: ['01-05-2024']
Mayor cantidad de ventas en una fecha: 3
Fecha/s con mayor monto vendido: ['10-05-2024']
Mayor monto vendido en una fecha: 350.0


Número de sucursal: 2
Monto total: 510.0
Producto más vendido: Producto C
Fecha/s con mayor cantidad de ventas: ['02-05-2024', '04-05-2024', '06-05-2024']
Mayor cantidad de ventas en una fecha: 1
Fecha/s con mayor monto vendido: ['02-05-2024']
Mayor monto vendido en una fecha: 130.0




#Conclusiones

## Parte 1

Luego de codificar la solución propuesta en distintos lenguajes pudimos notar diferencias entre ellos y además surgieron inconvenientes.

Por ejemplo, al codificar la solución en Python tuvimos conflicto con la identación, eso nos llevo a que el código no se ejecute correctamente y a obtener resultados inesperados. Por otra parte, en principio se desarrollo el código utilizando la biblioteca de Python llamada multiprocessing, que proporciona una interfaz de más alto nivel en comparación con la utilización de os.fork que es una interfaz de más bajo nivel. Respecto a esto una dificultad que encontramos es que fork no se puede ejecutar en Windows, mientras que multiprocessing si se puede. En este caso, preferimos utilizar os.fork ya que en el ejemplo dado en clase se utiliza os.fork, pero según lo que pudimos investigar solo funciona para Mac OS X, Unix y Linux, lo cual podría ser una desventaja ya que no asegura la portabilidad.

En el caso de Java, es un lenguaje de programación que tiene una sintaxis más compleja en comparación con Python, además de requerir más código para lograr el mismo resultado. Además, la ausencia de fork() puede presentar algunas limitaciones en la flexibilidad y eficiencia en comparación con C++, especialmente en escenarios donde la clonación exacta de un proceso es necesaria (cabe aclarar que aunque el espacio de memoria del proceso hijo es idéntico inicialmente al del proceso padre, cada uno tendrá su propia área de memoria y no se compartirá). Es por ello que Java ofrece alternativas como la clase ProcessBuilder, la cual permite configurar y ejecutar procesos externos, lo que facilita la interacción con programas externos y la comunicación a través de la entrada y salida estándar. Por otra parte, Java puede consumir más recursos en comparación con lenguajes de bajo nivel debido a la sobrecarga asociada con la JVM y la gestión de hilos, afectando de esta manera el rendimiento.

Durante el desarrollo en Rust, nos enfrentamos a varias dificultades, siendo una de las primeras la incorporación de la biblioteca externa "Nix" al programa. Esta biblioteca proporciona una alternativa de nivel superior para realizar operaciones con. La solución a este problema fue aprovechar del gestor de paquetes oficial de Rust, 'Cargo'. Al crear un proyecto en Cargo y agregar el código correspondiente, la incorporación de la biblioteca externa resultó relativamente sencilla. Por lo tanto, podemos afirmar que Cargo se convierte en una herramienta esencial para el desarrollo en este lenguaje.

Otro aspecto importante que debimos tener en cuenta fue la sintaxis particular del lenguaje. Aunque Rust está diseñado para que el código sea legible y seguro, al obligar al programador a ser más explícito, puede presentar una curva de aprendizaje más pronunciada para aquellos que no están familiarizados con él. Rust toma inspiración de la sintaxis de varios otros lenguajes de programación, como JavaScript, Python, C++ y Java.

Una de las particularidades que nos encontramos al desarrollar el código en GO es que no existe una sentencia similar a **fork()** , como las que conocimos en C y C++ en anteriores cursadas.

Para poder realizar programación concurrente en Go, utilizamos una característica sencilla propia del lenguaje que son la ejecucion de comandos. Asi es posible ejecutar varias tareas en forma concurrente sin preocuparse de la sobrecarga de recursos.
Con el *Cmd.Start()* es el método que inicia el comando especificado y no espera a que se complete.
La forma de utilizar los comandos es:
*cmd := exec.Command()*


## Parte 2

A partir del desarrollo realizado en C++ y la correspondiente toma de tiempos para distintos casos, podemos concluir que el llenado concurrente, especialmente con múltiples hilos, puede ser beneficioso para tamaños de matriz más grandes, ya que permite aprovechar mejor los recursos de la CPU y distribuir la carga de trabajo de manera más eficiente. Sin embargo, para tamaños de matriz pequeños y medianos, los beneficios pueden no existir o ser mínimos, debido a los costos adicionales asociados con la creación y administración de hilos, y la pequeña cantidad de trabajo que se realiza en estos tamaños de matrices.

Respecto a lo experimentado con Rust podemos concluir que para matrices pequeñas es contraproducente utilizar hilos ya que tenemos mayor tiempo de ejecución. Por ejemplo para una matriz de 1000x1000 vemos que la ejecucion secuencial toma
27611 ms mientras que la ejecucion que utiliza hilos demora 7415 ms. Entonces, concluimos que para matrices pequeñas recomendamos un único proceso pesado, mientras que para matrices más grandes conviene utilizar hilos ya que podemos encontrar una optimización de los tiempos de ejecución.

En GO funciona practicamente linealmente, una mejora buena pero no la mejor y soluciones lo suficientemente rapidas para obtener resultado con valores grandes y sencillo de programar. Pero no es la mejor ni la peor solucion de todas las mostradas.

En el caso de Java se puede observar que por mas que hay aun mejora de rendimiento mas evidente a medida que se utilizan matrices de mayor tamaño, hay un limite en el cual el numero de hilos se empieza a volver perjudicial para la velocidad del mismo. Siendo el sweetspot en este caso el usar no mas de 5 hilos. A partir de ese número, mas alla de si los procesadores son capaces de ejecutar esa cantidad de hilos o no, el overhead se vuelve demasiado pesado como para poder considerarlo. Aunque este número idea deberia cambiar si se utilizaran matrices con un tipo de dato mas pesado que un número.

En el caso de Python se puede observar que existe una mejor performance utilizando concurrencia con hilos. Con los datos obtenidos en todos los casos se ve una mejor performance en la ejecucion secuencial.


## Parte 3

Luego de desarrollar el ejercicio relativo a Python, podemos decir que surgieron pocas dificultades y pudieron ser resueltas: Por un lado, sucedió lo mismo que en la Parte 1 con el uso de os.fork(), lo cual no se puede ejecutar en Windows pero sí en Mac OS X, Unix y Linux, por lo que su funcionamiento debió ser probado directamente en este Colab. Como comentamos la vez anterior, esto significa una desventaja ya que no asegura la portabilidad. Y por otro lado, sucedió que en el primer fork (Padre A - Hijo B) cerrábamos los extremos del pipe correspondientes a la comunicación entre A y B en su debido lugar y momento, pero también los de la comunicación entre A y C, por lo que luego tuvimos problemas cuando se querían comunicar estos dos últimos procesos ya que encontraban sus extremos cerrados. Una vez detectado, también fue resuelto.